In [ ]:
!pip install pymongo

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
!pip freeze

pymongo==4.1.1


In [ ]:
import pymongo
import json
from datetime import datetime
import pandas as pd

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/") 
dbnames = client.list_database_names()
db = client["TweetsDB"]
tweets_coll = db["Tweets_data"]
print("Database connection established")

Database connection established


In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/") 
dbnames = client.list_database_names()
if "TweetsDB" in dbnames:
    print("db exists. Will be deleted...")
    client.drop_database("TweetsDB")
db = client["TweetsDB"]
col_names = db.list_collection_names()
if "Tweets_data" in col_names:
        print("Tweets Collection exists. Will be deleted...")
        db.Tweets_data.drop()
tweets_coll = db["Tweets_data"]
print("Database connection established")

db exists. Will be deleted...
Database connection established


In [ ]:
def insert_into_mongo(dict):
    db.tweets_coll.insert_one(dict)

In [ ]:
with open('C:/Users/widok/Downloads/Database Management Systems/corona-out-2/corona-out-2', "r") as read_file:
        json_data = json.load(read_file)

In [ ]:
def find_in_collection(tid):
    record = None
    record = db.tweets_coll.find({'TweetID': tid})
    if len(list(record)) > 0:
        return True
    return False

In [ ]:
def increment_retweet_count(tid):
    db.tweets_coll.update_one({'TweetID': tid}, {'$inc': {'Retweet_Count': 1}})

In [ ]:
for i in range(1,len(json_data)):
    
#    try:
        
    if find_in_collection(json_data[i]['id']):
        continue

    if json_data[i]['text'].startswith('RT'):
        if 'retweeted_status' in json_data[i]:
            RetweetID = json_data[i]['retweeted_status']['id']
            if find_in_collection(RetweetID):
                increment_retweet_count(RetweetID)
            else:
                dict = {
                'created_at':datetime.strftime(datetime.strptime(json_data[i]['retweeted_status']['created_at'],'%a %b %d %H:%M:%S +0000 %Y'),'%Y-%m-%d %H:%M:%S'),
                'TweetID':json_data[i]['retweeted_status']['id'],
                'Id_str':json_data[i]['retweeted_status']['id_str'],
                'Text':json_data[i]['retweeted_status']['text'],
                'Hashtag':list(map(lambda x: x["text"], json_data[i]['retweeted_status']['entities']['hashtags'])),
                'UserID':json_data[i]['retweeted_status']['user']['id'],
                'Retweet_Count': 1,
                'Retweet_ID': 0
                }

                insert_into_mongo(dict)
    else:
        RetweetID = 0


    dict = {
        'created_at':datetime.strftime(datetime.strptime(json_data[i]['created_at'],'%a %b %d %H:%M:%S +0000 %Y'),'%Y-%m-%d %H:%M:%S'),
        'TweetID':json_data[i]['id'],
        'Id_str':json_data[i]['id_str'],
        'Text':json_data[i]['text'],
        'Hashtag':list(map(lambda x: x["text"], json_data[i]['entities']['hashtags'])),
        'UserID':json_data[i]['user']['id'],
        'Retweet_Count': 0,
        'Retweet_ID':RetweetID
    }

    insert_into_mongo(dict)                   
            
    
#    except:
#        continue

In [ ]:
pd.DataFrame(db.tweets_coll.find({}).limit(1))

,Hashtag,Id_str,Retweet_Count,Retweet_ID,Text,TweetID,UserID,_id,created_at
0,[],1254022770679320576,0,0,"É isto, ou vou morrer sem ar ou com o corona h...",1254022770679320576,804046791348015107,62696f3605a34a4e6308a367,2020-04-25 12:21:41


### Tweets within daterange

In [ ]:
def find_tweets_in_daterange(start_date,end_date):
    agg_result = db.tweets_coll.aggregate(
        [
           {"$match":{"created_at": { "$gte": start_date,"$lte": end_date }} },
           {"$limit":2}
        ]
    )
    
    for doc in agg_result:
        print(doc)

In [ ]:
find_tweets_in_daterange("2016-11-30","2016-12-30")

{'_id': ObjectId('62687eeae183d99280c6118b'), 'created_at': '2016-12-29 03:47:01', 'TweetID': 1254022787196387329, 'Id_str': '1254022787196387329', 'Text': 'RT @PassionFruit62: The Trump Admin bought at least 29 mil doses of this poison and promoted it daily through press briefings, \u2066@seanhannit…', 'Hashtag': [], 'UserID': 814316767665418248, 'Retweet_Count': 4, 'Retweet_ID': 1254018062694703106}
{'_id': ObjectId('62687eeae183d99280c611e6'), 'created_at': '2016-12-10 17:38:00', 'TweetID': 1254022845233016832, 'Id_str': '1254022845233016832', 'Text': 'RT @lewinskylou2: SCHATTING WERKELIJK AANTAL CORONA-DODEN T/M 25 APRIL\nWerkelijk aantal = 2 x bevestigd aantal\n\n2 x 4.409 = 8.818 overleden…', 'Hashtag': [], 'UserID': 807640520646217728, 'Retweet_Count': 6, 'Retweet_ID': 1254020595920113664}


### Search with hastags within a daterange

In [ ]:
def hastags_in_daterange(start_date,end_date):
    agg_result = db.tweets_coll.aggregate([
     {
      "$match":{
        "$and":[
              {'Hashtag': { '$elemMatch': { '$eq': 'corona'} }},
              {"created_at": { "$gte": start_date,"$lte": end_date }}
        ]
       }},

        {"$limit":2}
    ])

    for doc in agg_result:
            print(doc)

In [ ]:
hastags_in_daterange("2016-11-30","2016-12-30")

{'_id': ObjectId('62687f08e183d99280c6ba34'), 'created_at': '2016-12-08 03:13:54', 'TweetID': 1254049455462068224, 'Id_str': '1254049455462068224', 'Text': 'RT @OKAZ_online: الوطن بخير\n#عكاظ #ان_تكون_اولا #السعودية #كورونا #رمضان #corona\nhttps://t.co/3SlA4INXWI https://t.co/gZ0wHwvWoK', 'Hashtag': ['عكاظ', 'ان_تكون_اولا', 'السعودية', 'كورونا', 'رمضان', 'corona'], 'UserID': 806698288045035520, 'Retweet_Count': 22, 'Retweet_ID': 1253815812785418240}
{'_id': ObjectId('62687f11e183d99280c6fb34'), 'created_at': '2016-12-10 17:38:00', 'TweetID': 1254059034312876033, 'Id_str': '1254059034312876033', 'Text': 'RT @delangemars: LIVE Protest tegen #corona dictatuur in #Berlin van @Ruptly bekijken\n\n#wirsinddasvolk https://t.co/qngB8YInJ0', 'Hashtag': ['corona', 'Berlin', 'wirsinddasvolk'], 'UserID': 807640520646217728, 'Retweet_Count': 5, 'Retweet_ID': 1254054072992309249}


### Indexing

In [ ]:
pd.DataFrame(db.tweets_coll.index_information())

,_id_
key,"[(_id, 1)]"
v,2


In [ ]:
db.tweets_coll.create_index("TweetID")
db.tweets_coll.create_index("UserID")
db.tweets_coll.create_index("created_at")
db.tweets_coll.create_index([("Text", "text")], name = 'tweet_text')

'tweet_text'

In [ ]:
pd.DataFrame(db.tweets_coll.list_indexes())

,default_language,key,language_override,name,textIndexVersion,v,weights
0,NaN,{'_id': 1},NaN,_id_,NaN,2,NaN
1,NaN,{'TweetID': 1},NaN,TweetID_1,NaN,2,NaN
2,NaN,{'UserID': 1},NaN,UserID_1,NaN,2,NaN
3,NaN,{'created_at': 1},NaN,created_at_1,NaN,2,NaN
4,english,"{'_fts': 'text', '_ftsx': 1}",language,tweet_text,3.0,2,{'Text': 1}


### Top 10 tweets